In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from get_lat_lon import get_lat_lon_from_address

In [4]:
# Read recipe inputs
raw_ramen = dataiku.Dataset("ramen_by_store_name")
df = raw_ramen.get_dataframe()

In [5]:
df.head()

,store_name,score,review_cnt,store_id,address,ward,review
0,81番,3.70,205,41,東京都町田市原町田4-5-18仲見世商店街,東京都内,どうもMormorです今はTwitter情報でリニューアル前の番のラーメン洋をイメージした限...
1,BASSO 江戸川橋,3.68,137,33,東京都新宿区山吹町130-15,東京都内,金曜の時半過ぎに到着。並ばず入れました。はじめ煮干そばの特製円税込也を購入。着席後限定の存在...
2,Homemade Ramen 麦苗,3.97,731,2,東京都品川区南大井6-11-10,東京都内,度目の訪問現在は蜜を避けるために事前記帳予約式になっています受け付けは時から各時間帯人の枠が...
3,Japanese Soba Noodles 蔦,3.81,220,8,東京都渋谷区西原3-2-4フロンティア代々木上原 B1F,東京都内,ラーメン部門でミシュランの一つ星を初めて獲得したお店だそうです。移転前の巣鴨で時間以上待った...
4,MENSHO,3.82,369,3,東京都文京区音羽1-17-16中銀音羽マンシオン１F,東京都内,前回つけ麺の麺の量を知らずせっかくの美味なスープを残して麺が早々と終わってしまったので今回は...


In [8]:
addresses = df['address'].tolist()

In [0]:
lat_lons = get_lat_lon_from_address(addresses)

  0%|          | 1/564 [00:13<2:10:01, 13.86s/it]

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

lat_lon_df = raw_ramen_df # For this sample code, simply copy input to output


# Write recipe outputs
lat_lon = dataiku.Dataset("lat_lon")
lat_lon.write_with_schema(lat_lon_df)